In [1]:
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.metrics import jaccard_score
import pandas as pd
import re
from tqdm import tqdm
import tifffile

# PlantSeg imports
import plantseg
from plantseg.core.zoo import ModelZoo
from plantseg.tasks import import_image_task, unet_prediction_task

# CellPose imports
from cellpose import models, io

import napari


from pathlib import Path
from plantseg.tasks import import_image_task, unet_prediction_task
from plantseg.core import PlantSegImage, ImageProperties, image
import cv2
from plantseg.core.zoo import ModelZoo

mz = ModelZoo(plantseg.PATH_MODEL_ZOO, plantseg.PATH_MODEL_ZOO_CUSTOM)
plantseg_model_names = mz.get_model_names()

from cellpose import models, utils, io
import matplotlib.pyplot as plt
import cellpose.models

cp_models = ["cyto3", "nuclei", "cyto2_cp3", "tissuenet_cp3", "livecell_cp3", "yeast_PhC_cp3", "yeast_BF_cp3", "bact_phase_cp3", "bact_fluor_cp3", "deepbacs_cp3", "cyto2", "cyto", "CPx", "transformer_cp3", "neurips_cellpose_default", "neurips_cellpose_transformer", "neurips_grayscale_cyto2", "CP", "CPx", "TN1", "TN2", "TN3", "LC1", "LC2", "LC3", "LC4"]

ps_models = [
    "generic_confocal_3D_unet",
    "generic_light_sheet_3D_unet",
    "confocal_3D_unet_ovules_ds1x",
    "confocal_3D_unet_ovules_ds2x",
    "confocal_3D_unet_ovules_ds3x",
    "confocal_2D_unet_ovules_ds2x",
    "lightsheet_3D_unet_root_ds1x",
    "lightsheet_3D_unet_root_ds2x",
    "lightsheet_3D_unet_root_ds3x",
    "lightsheet_2D_unet_root_ds1x",
    "lightsheet_3D_unet_root_nuclei_ds1x",
    "lightsheet_2D_unet_root_nuclei_ds1x",
    "confocal_2D_unet_sa_meristem_cells",
    "confocal_3D_unet_sa_meristem_cells",
    "lightsheet_3D_unet_mouse_embryo_cells",
    "confocal_3D_unet_mouse_embryo_nuclei",
    "PlantSeg_3Dnuc_platinum",
]

import warnings

# Suppress PyTorch warnings
warnings.filterwarnings("ignore", message="You are using `torch.load`", category=FutureWarning)

INFO: P [MainThread] 2025-01-31 12:31:49,677 plantseg - Logger configured at initialisation. PlantSeg logger name: plantseg


In [2]:
viewer = napari.Viewer()

In [3]:
# Define the function to construct the dataset DataFrame
def construct_dataset_dataframe(base_dir):
    columns = ["dataset_name", "sequence_name", "image_path", "mask", "gold_mask"] + cp_models + ps_models

    data = []

    for dataset_name in os.listdir(base_dir):
        dataset_path = os.path.join(base_dir, dataset_name)
        if not os.path.isdir(dataset_path):
            continue

        for sequence_name in os.listdir(dataset_path):
            sequence_path = os.path.join(dataset_path, sequence_name)
            if not os.path.isdir(sequence_path) or "_" in sequence_name:
                continue

            # Paths for masks and gold masks
            err_seg_path = os.path.join(dataset_path, f"{sequence_name}_ERR_SEG")
            gt_seg_path = os.path.join(dataset_path, f"{sequence_name}_GT", "SEG")

            for image_file in os.listdir(sequence_path):
                if image_file.endswith(".tif"):
                    image_path = os.path.join(sequence_path, image_file)

                    # Corresponding mask and gold mask paths
                    mask_path = os.path.join(err_seg_path, f"mask{image_file[1:]}")
                    mask_path = mask_path if os.path.exists(mask_path) else None

                    gold_mask_path = os.path.join(gt_seg_path, f"man_seg{image_file[1:]}")
                    gold_mask_path = gold_mask_path if os.path.exists(gold_mask_path) else None

                    # Append a row with default None values for models
                    data.append([dataset_name, sequence_name, image_path, mask_path, gold_mask_path] + [None] * (len(columns) - 5))

    return pd.DataFrame(data, columns=columns)


# Define the base directory (update this to your actual dataset path)
base_dir = "./datasets"

# Create the DataFrame
dataset_df = construct_dataset_dataframe(base_dir)

# Save or display the DataFrame
# print(dataset_df.head())
# Optionally save to a CSV file
dataset_df.to_csv("dataset_summary.csv", index=False)

In [4]:
# Helper function for loading images
def load_image(file_path):
    return cv2.imread(file_path, cv2.IMREAD_UNCHANGED)


# # Function to perform segmentation using PlantSeg
# def segment_with_plantseg(image_path, model_name="lightsheet_2D_unet_root_ds1x", patch=(1, 64, 64), device="cuda"):
#     plantseg_image = import_image_task(input_path=Path(image_path), semantic_type="raw", stack_layout="YX")
#     predicted_images = unet_prediction_task(image=plantseg_image, model_name=model_name, patch=patch, device=device)
#     # Return the first predicted image
#     return predicted_images[0].get_data()


# # Function to perform segmentation using CellPose
# def segment_with_cellpose(image_path, model_type="cyto2", flow_threshold=0.4, cellprob_threshold=0):
#     image = io.imread(image_path)
#     model = models.Cellpose(gpu=True, model_type=model_type)
#     masks, flows, styles, diams = model.eval(image, diameter=None, channels=[0, 0], flow_threshold=flow_threshold, cellprob_threshold=cellprob_threshold)
#     return masks


# Function for calculating IoU with ground truth
def calculate_iou(pred, gt):
    pred_flat = pred.flatten()
    gt_flat = gt.flatten()
    return jaccard_score(gt_flat, pred_flat, average="binary")


def get_images_for_masks(gt_gold_paths, image_paths):
    subset_paths = []
    for p in gt_gold_paths:
        first_gt_gold_file = os.path.basename(p)
        # Extract digits from the filename
        digits = int(re.findall(r"\d+", first_gt_gold_file)[0])
        subset_paths.append(image_paths[digits])
    return subset_paths


# def segment(image_path, model_name):
#     if model_name in cp_models:
#         return segment_with_cellpose(image_path, model_type=model_name)
#     elif model_name in ps_models:
#         return segment_with_plantseg(image_path, model_name=model_name)
#     else:
#         raise ValueError(f"Unknown model name: {model_name}")


# def seg_with_CellPose(image_path, model_type="cyto2", diameter=None):
#     image = io.imread(image_path)
#     model = models.CellposeModel(gpu=True, model_type=model_type)
#     masks, flows, styles = model.eval(image, diameter=diameter, channels=[0, 0], flow_threshold=0.4, cellprob_threshold=0)
#     return masks


# def seg_with_PlantSeg(image_path, model_name="lightsheet_2D_unet_root_ds1x", patch=(1, 64, 64), device="cuda"):
#     plantseg_image = import_image_task(input_path=Path(image_path), semantic_type="raw", stack_layout="YX")
#     predicted_images = unet_prediction_task(image=plantseg_image, model_name=model_name, patch=patch, device=device)
#     return predicted_images[0].get_data()


def ins_to_sem(mask):
    # Convert instance segmentation mask to semantic mask
    mask = mask.astype(np.uint8)
    mask = mask * 255
    mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    return mask


# def calc_jaccard_score(pred, gt):
#     pred_flat = pred.flatten()
#     gt_flat = gt.flatten()
#     return jaccard_score(gt_flat, pred_flat, average="micro")


# def calc_jaccard_score_per_object(pred, gt):
#     # Unique labels for reference objects (ground truth) and predicted objects
#     gt_labels = np.unique(gt)
#     pred_labels = np.unique(pred)

#     # Remove background label (assumed to be 0)
#     gt_labels = gt_labels[gt_labels > 0]
#     pred_labels = pred_labels[pred_labels > 0]

#     jaccard_scores = []

#     for gt_label in gt_labels:
#         # Extract pixels for the current reference object
#         gt_object = gt == gt_label

#         # Find the matching predicted object with maximum overlap
#         best_iou = 0
#         for pred_label in pred_labels:
#             pred_object = pred == pred_label
#             intersection = np.logical_and(gt_object, pred_object).sum()
#             union = np.logical_or(gt_object, pred_object).sum()

#             if union > 0:
#                 iou = intersection / union
#                 best_iou = max(best_iou, iou)

#         # Append the best IoU for this ground truth object
#         jaccard_scores.append(best_iou)

#     # Mean IoU across all ground truth objects
#     return np.mean(jaccard_scores) if jaccard_scores else 0

In [5]:
# for index, row in dataset_df.iterrows():

#     for model in cp_models + ps_models:
#         image_path = row["image_path"][0]  # Assuming you want to use the first image in the list
#         gt_path = row["gt_path"][0]  # Assuming you want to use the first ground truth in the list
#         gt_gold_path = row["gt_gold_path"][0]  # Assuming you want to use the first ground truth gold in the list

#         # Perform segmentation
#         segmented_image = segment(image_path, model_name=model)

#         # Convert instance segmentation to semantic segmentation
#         semantic_mask = ins_to_sem(segmented_image)

#         # Load ground truth
#         gt_image = load_image(gt_gold_path)
#         gt_semantic = ins_to_sem(gt_image)

#         # Calculate IoU score
#         iou_score = calc_jaccard_score_per_object(semantic_mask, gt_semantic)
#         print(f"Model: {model}, IoU Score: {iou_score}")

In [6]:
# Function to perform segmentation
def segment(image_path, model_name, dimension="2D"):
    twoD_ps_models = [model for model in ps_models if "2D" in model]
    threeD_ps_models = [ps for ps in ps_models if "3D" in ps]
    if model_name in cp_models:
        model = models.CellposeModel(gpu=True, model_type=model_name)
        image = io.imread(image_path)
        masks, _, _ = model.eval(image, diameter=None, channels=[0, 0], flow_threshold=0.4, cellprob_threshold=0, do_3D=True if len(image.shape) == 3 else False)
        return masks
    elif model_name in twoD_ps_models:
        plantseg_image = import_image_task(input_path=Path(image_path), semantic_type="raw", stack_layout="YX")
        predicted_images = unet_prediction_task(image=plantseg_image, model_name=model_name, model_id=None, patch=(1, 64, 64), device="cuda")
        return predicted_images[0].get_data()
    elif model_name in threeD_ps_models:
        plantseg_image = import_image_task(input_path=Path(image_path), semantic_type="raw", stack_layout="ZYX")
        predicted_images = unet_prediction_task(image=plantseg_image, model_name=model_name, model_id=None, patch=(4, 64, 64), device="cuda")
        return predicted_images[0].get_data()
    else:
        raise ValueError(f"Unknown model name: {model_name}")


# # Function to calculate Jaccard score
# def calculate_jaccard_score(predicted_mask, ground_truth_mask):
#     # Ensure both masks are the same data type
#     pred_flat = predicted_mask.flatten().astype(np.int32)
#     gt_flat = ground_truth_mask.flatten().astype(np.int32)
#     return jaccard_score(gt_flat, pred_flat, average="micro")


def calculate_jaccard_score(predicted_mask, ground_truth_mask):
    """
    Calculate the Jaccard Index (IoU) for 2D or 3D masks.

    Parameters:
    - predicted_mask: np.array, predicted mask of shape (X, Y) or (Z, X, Y).
    - ground_truth_mask: np.array, ground truth mask of shape (X, Y) or (Z, X, Y).

    Returns:
    - jaccard_score: float, IoU for 2D data or averaged IoU for 3D data.
    """
    # Ensure both masks have the same shape
    assert predicted_mask.shape == ground_truth_mask.shape, f"Masks must have the same shape. but they're pred:{predicted_mask.shape} and gt:{ground_truth_mask.shape}"

    # Flatten and calculate IoU for 2D data
    if predicted_mask.ndim == 2:
        pred_flat = predicted_mask.flatten().astype(np.int32)
        gt_flat = ground_truth_mask.flatten().astype(np.int32)
        return jaccard_score(gt_flat, pred_flat, average="weighted")

    # Process 3D data (compute IoU for each slice, then average)
    elif predicted_mask.ndim == 3:
        jaccard_scores = []
        for z in range(predicted_mask.shape[0]):  # Iterate through slices
            pred_flat = predicted_mask[z].flatten().astype(np.int32)
            gt_flat = ground_truth_mask[z].flatten().astype(np.int32)
            jaccard_scores.append(jaccard_score(gt_flat, pred_flat, average="weighted"))
        return np.mean(jaccard_scores)  # Average over all slices

    else:
        raise ValueError("Input masks must be 2D or 3D arrays.")


# Main processing loop
def process_dataset(df, models, original_df=None):
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing dataset", position=0):
        try:
            image_path = row["image_path"]
            mask_path = row["mask"] if pd.notna(row["mask"]) else row["gold_mask"]

            assert image_path is not None
            assert mask_path is not None

            if not mask_path or not os.path.exists(mask_path):
                continue

            ground_truth_mask = tifffile.imread(mask_path)

            for model_name in models:
                if pd.notna(row.get(model_name)):
                    print(f"skipping {row['dataset_name']}")
                    continue  # Skip if Jaccard is already calculated
                try:
                    print(f"Processing dataset {row['dataset_name']} with model {model_name}")
                    predicted_mask = segment(image_path, model_name)
                    iou_score = calculate_jaccard_score(predicted_mask, ground_truth_mask)
                    df.at[idx, model_name] = iou_score
                    if original_df is not None:
                        original_df.loc[df.index[idx], model_name] = iou_score
                    print("JACCARD SCORE IS:", iou_score)
                except Exception as e:
                    print(f"Error processing model {model_name} for image {image_path}: {e}")
        except Exception as e:
            # print(f"Error processing row {idx} for model {model_name}: {e}")
            print(image_path)
            print(mask_path)

In [7]:
from typing import Tuple


def load_datasets_from_csv(dd_path=r"./sample_summary.csv", ddd_path=r"./sample3D_summary.csv") -> Tuple[pd.DataFrame, pd.DataFrame]:
    return pd.read_csv(dd_path), pd.read_csv(ddd_path)

# 2D Datasets

In [9]:
# dataset_df = pd.read_csv("dataset_summary.csv")

In [22]:
sampled_df = pd.read_csv("sample_summary.csv")

In [ ]:
sampled_df.head(1)

In [10]:
filter_df = dataset_df.dropna(subset=["mask", "gold_mask"], how="all")
# sampled_df = filter_df.groupby(["dataset_name", "sequence_name"], group_keys=False).apply(lambda x: x.sample(n=min(5, len(x)), random_state=42)).reset_index(drop=True)

In [ ]:
the_models = [f for f in ps_models if "2D" in f]
the_models

In [12]:
# process_dataset(sampled_df, the_models)

In [ ]:
sampled_df

In [90]:
# sampled_df.to_csv("sample3D_summary.csv", index=True)

In [8]:
# dataset_df.to_csv("dataset_summary.csv", index=False)

In [ ]:
image_path = sampled_df["image_path"][0]
image_path

In [ ]:
plantseg_image = import_image_task(input_path=Path(image_path), semantic_type="raw", stack_layout="ZYX")

In [ ]:
predicted_images = unet_prediction_task(image=plantseg_image, model_name=the_models[0], model_id=None, patch=(1, 64, 64), device="cuda")
# Return the first predicted image
plt.imshow(predicted_images[0].get_data())

In [ ]:
# Columns representing the models
model_columns = cp_models + ps_models

# Group by dataset and sequence, then calculate the mean IoU for each model
average_iou = sampled_df.groupby(["dataset_name", "sequence_name"])[model_columns].mean().reset_index()

# Calculate overall average IoU across datasets
overall_average_iou = average_iou[model_columns].mean().to_dict()

average_iou, overall_average_iou

In [ ]:
overall_average_iou

# 3D

In [8]:
sampled_3D_df = pd.read_csv("sample3D_summary.csv")

In [8]:
filter_df = dataset_df.dropna(subset=["mask", "gold_mask"], how="all")

In [ ]:
filtered_3d_df = filter_df[filter_df["dataset_name"].str.contains("3D")]
filtered_3d_df.head(1)

In [ ]:
sampled_3D_df = filtered_3d_df.groupby(["dataset_name", "sequence_name"], group_keys=False).apply(lambda x: x.sample(n=min(1, len(x)), random_state=42)).reset_index(drop=True)

In [ ]:
# Check if the values under dataset_name in sampled_3D_df and sampled_df are the same
same_values = sampled_3D_df["dataset_name"].equals(sampled_df["dataset_name"])
print(same_values)

In [ ]:
sampled_3D_df.head(1)

In [ ]:
# process_dataset(sampled_3D_df, cp_models[15:])
process_dataset(sampled_3D_df, ["generic_confocal_3D_unet"])

In [18]:
sampled_3D_df.to_csv("sample3D_summary.csv", index=False)

In [ ]:
image_path = sampled_3D_df["image_path"][5]
image_path
plantseg_image = import_image_task(input_path=Path(image_path), semantic_type="raw", stack_layout="ZYX")
predicted_images = unet_prediction_task(image=plantseg_image, model_name="generic_confocal_3D_unet", model_id=None, patch=(4, 64, 64), device="cuda")
# Return the first predicted image
# plt.imshow(predicted_images[0].get_data())

In [21]:
pred_mask = predicted_images[0].get_data()

In [19]:
viewer = napari.Viewer()

In [ ]:
viewer.add_image(predicted_images[0].get_data())

In [22]:
mask_path = sampled_3D_df["mask"][0]
import tifffile

mask = tifffile.imread(mask_path)
# viewer.add_image(mask)

In [20]:
# import numpy as np


# def calculate_jaccard_index(predicted_mask, ground_truth_mask):
#     """
#     Calculate the Jaccard Index (IoU) for 3D masks.

#     Parameters:
#     - predicted_mask: np.array, predicted mask of shape (Z, X, Y). Can be binary or multi-class.
#     - ground_truth_mask: np.array, ground truth mask of shape (Z, X, Y). Can be binary or multi-class.

#     Returns:
#     - jaccard_index: float or dict, IoU score(s) for binary or per class if multi-class.
#     """
#     # Ensure masks are the same shape
#     assert predicted_mask.shape == ground_truth_mask.shape, "Masks must have the same shape."

#     # Convert masks to a consistent binary format (np.bool_)
#     predicted_mask = np.asarray(predicted_mask, dtype=bool)
#     ground_truth_mask = np.asarray(ground_truth_mask, dtype=bool)

#     if np.array_equal(np.unique(predicted_mask), [False, True]) and np.array_equal(np.unique(ground_truth_mask), [False, True]):
#         # Binary case
#         intersection = np.logical_and(predicted_mask, ground_truth_mask)
#         union = np.logical_or(predicted_mask, ground_truth_mask)
#         return np.sum(intersection) / np.sum(union) if np.sum(union) > 0 else 1.0  # Handle zero union case

#     else:
#         # Multi-class case
#         classes = np.unique(ground_truth_mask)
#         iou_scores = {}
#         for cls in classes:
#             pred_binary = predicted_mask == cls
#             gt_binary = ground_truth_mask == cls
#             intersection = np.logical_and(pred_binary, gt_binary)
#             union = np.logical_or(pred_binary, gt_binary)
#             iou_scores[cls] = np.sum(intersection) / np.sum(union) if np.sum(union) > 0 else 1.0
#         return iou_scores

In [ ]:
# Example usage

jaccard_index = calculate_jaccard_score(pred_mask, mask)
print("Jaccard Index:", jaccard_index)

In [ ]:
sampled_3D_df.head(1)

In [15]:
ps_3d_models = [ps for ps in ps_models if "3D" in ps]

In [ ]:
process_dataset(sampled_3D_df, cp_models)

In [28]:
sampled_3D_df.to_csv("sample3D_summary.csv", index=False)

In [ ]:
model = "deepbacs_cp3"
image_path = sampled_3D_df["image_path"][11]
gt_mask = sampled_3D_df["mask"][11]
image_path

In [ ]:
mask = segment(image_path, "cyto3")

In [63]:
img = tifffile.imread(image_path)
gt_mask = tifffile.imread(gt_mask)

In [ ]:
# i = 34
plt.imshow(gt_mask[i])

In [ ]:
plt.imshow(mask[i])

In [ ]:
jac = calculate_jaccard_score(mask[i], gt_mask[i])
jac

# Re-doing for missing values

In [ ]:
model = "cyto2_cp3"
missing_df = sampled_3D_df[sampled_3D_df["dataset_name"] == "Fluo-C3DH-A549-SIM"]
missing_df

In [ ]:
process_dataset(sampled_3D_df, ["generic_confocal_3D_unet"])

In [38]:
sampled_3D_df.to_csv("sample3D_summary.csv", index=False)

In [8]:
sampled_df, sampled_3D_df = load_datasets_from_csv()

In [9]:
sampled_3D_df

,dataset_name,sequence_name,image_path,mask,gold_mask,cyto3,nuclei,cyto2_cp3,tissuenet_cp3,livecell_cp3,...,lightsheet_3D_unet_root_ds2x,lightsheet_3D_unet_root_ds3x,lightsheet_2D_unet_root_ds1x,lightsheet_3D_unet_root_nuclei_ds1x,lightsheet_2D_unet_root_nuclei_ds1x,confocal_2D_unet_sa_meristem_cells,confocal_3D_unet_sa_meristem_cells,lightsheet_3D_unet_mouse_embryo_cells,confocal_3D_unet_mouse_embryo_nuclei,PlantSeg_3Dnuc_platinum
0,Fluo-C3DH-A549,1,./datasets/Fluo-C3DH-A549/01/t015.tif,./datasets/Fluo-C3DH-A549/01_ERR_SEG/mask015.tif,./datasets/Fluo-C3DH-A549/01_GT/SEG/man_seg015...,0.958339,0.955484,0.964972,0.955350,0.955350,...,0.955351,0.955350,NaN,0.955351,NaN,NaN,0.955351,NaN,0.955351,0.955351
1,Fluo-C3DH-A549,2,./datasets/Fluo-C3DH-A549/02/t015.tif,./datasets/Fluo-C3DH-A549/02_ERR_SEG/mask015.tif,./datasets/Fluo-C3DH-A549/02_GT/SEG/man_seg015...,0.965630,0.963048,0.966981,0.962783,0.962783,...,0.962784,0.962783,NaN,0.962784,NaN,NaN,0.962784,NaN,0.962784,0.962784
2,Fluo-C3DH-A549-SIM,1,./datasets/Fluo-C3DH-A549-SIM/01/t015.tif,./datasets/Fluo-C3DH-A549-SIM/01_ERR_SEG/mask0...,./datasets/Fluo-C3DH-A549-SIM/01_GT/SEG/man_se...,0.969004,0.970823,0.979676,0.970930,0.932429,...,0.970931,0.970931,NaN,0.970931,NaN,NaN,0.970931,NaN,0.970931,0.970931
3,Fluo-C3DH-A549-SIM,2,./datasets/Fluo-C3DH-A549-SIM/02/t015.tif,./datasets/Fluo-C3DH-A549-SIM/02_ERR_SEG/mask0...,./datasets/Fluo-C3DH-A549-SIM/02_GT/SEG/man_se...,0.967152,0.972035,0.955095,0.972234,0.972234,...,0.972234,0.972234,NaN,0.972234,NaN,NaN,0.972234,NaN,0.972234,0.972237
4,Fluo-C3DH-H157,1,./datasets/Fluo-C3DH-H157/01/t024.tif,./datasets/Fluo-C3DH-H157/01_ERR_SEG/mask024.tif,NaN,0.884743,0.883142,0.884054,0.882900,0.882900,...,0.882900,0.882900,NaN,0.882903,NaN,NaN,0.882900,NaN,0.882900,0.882900
5,Fluo-C3DH-H157,2,./datasets/Fluo-C3DH-H157/02/t024.tif,./datasets/Fluo-C3DH-H157/02_ERR_SEG/mask024.tif,NaN,0.979923,0.975333,0.970530,0.975754,0.975754,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Fluo-N3DH-CE,1,./datasets/Fluo-N3DH-CE/01/t189.tif,./datasets/Fluo-N3DH-CE/01_ERR_SEG/mask189.tif,NaN,0.821275,0.821269,0.820035,0.821279,0.819753,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Fluo-N3DH-CE,2,./datasets/Fluo-N3DH-CE/02/t187.tif,./datasets/Fluo-N3DH-CE/02_ERR_SEG/mask187.tif,NaN,0.775007,0.777089,0.775092,0.777094,0.777094,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Fluo-N3DH-CHO,1,./datasets/Fluo-N3DH-CHO/01/t030.tif,./datasets/Fluo-N3DH-CHO/01_ERR_SEG/mask030.tif,NaN,0.802870,0.802356,0.789845,0.802599,0.800809,...,0.802598,0.802598,NaN,0.802598,NaN,NaN,0.802598,NaN,0.802599,0.802587
9,Fluo-N3DH-CHO,2,./datasets/Fluo-N3DH-CHO/02/t030.tif,./datasets/Fluo-N3DH-CHO/02_ERR_SEG/mask030.tif,NaN,0.862758,0.869126,0.789845,0.881402,0.881475,...,0.881402,0.881400,NaN,0.881400,NaN,NaN,0.881402,NaN,0.881399,0.881357
